In [ ]:
import numpy as np
import pymc as pm
from scipy.stats import norm
from scipy import optimize
import scipy.linalg as la

def GetData():
    # load data from input.csv and output.csv
    # input.csv: model parameters used to generate the data (can be used to visualize samples vs actual parameters)
    # output.csv: "measured" natural frequencies from input
    input = np.genfromtxt("input.csv",delimiter=',')
    output = np.genfromtxt("output.csv",delimiter=',')
    return (input,output)

def NatFreqModel(t1,t2):

    k1 = t1
    k2 = 1500
    k3 = 1500
    k4 = t2
    
    m = 100
    
    # masses
    m1 = m
    m2 = 2*m
    m3 = 3*m
    
    M = np.array([[m1,0,0],[0,m2,0],[0,0,m3]])
    K = np.array([[k1+k2,-k2,0],[-k2,k2+k3,-k3],[0,-k3,k3+k4]])
    # some algorihtm to find the eigenvalues from (K-lambda*M) = 0 (I forgot the source sorry)
    # scipy.linalg.eig(K,M) would do the same, but then the order of the output is changed, so I left it at this
    a = -m1*m2*m3
    b = K[0,0]*m2*m3+m1*K[1,1]*m3 + m1*m2*K[2,2]
    c = -K[0,0]*K[1,1]*m3 - K[0,0]*K[2,2]*m2 - K[1,1]*K[2,2]*m1+K[1,2]*K[2,1]*m1+K[0,1]*K[1,0]*m3
    d = K[0,0]*K[1,1]*K[2,2]-K[0,0]*K[1,2]*K[2,1]-K[0,1]*K[1,0]*K[2,2]
    
    p = 3*a*c-b**2
    q = 2*b**3-9*a*b*c+27*a**2*d
    phi = np.arccos(-q/(2*np.sqrt(-p**3)))
    y = np.sqrt(-p)*2*np.cos(phi/3+np.array([0,1,2])*2*np.pi/(3))

    eigFreq = (y-b)/(3*a) # 1x3 array with the natural frequencies

    return eigFreq

In [ ]:
input, data = GetData() # load data

sigma_noise = 0.1 # Your actual noise standard deviation here

with pm.Model() as model:
    # Define priors on theta
    # theta = pm.Normal('theta', mu=0, sd=1, shape=n_theta)
    k1 = pm.Uniform('k1', lower=500, upper=3500)
    k4 = pm.Uniform('k4', lower=500, upper=3500)
    
#     # Define likelihood
    likelihood = pm.Normal('likelihood', NatFreqModel(k1,k4), sigma=sigma_noise, observed=data)

#     # Perform MCMC sampling
    trace = pm.sample(2000, tune=500, cores=2)


In [ ]:
# summarizing the posterior
pm.summary(trace)

# plotting the posterior
pm.plot_trace(trace)

pm.plot_pair(trace)